In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
%matplotlib inline

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def readTrain():
  train = pd.read_csv("S&P 500.csv")
  return train

In [3]:
def augFeatures(train):
  train["Date"] = pd.to_datetime(train["Date"])
  train["year"] = train["Date"].dt.year
  train["month"] = train["Date"].dt.month
  train["date"] = train["Date"].dt.day
  train["day"] = train["Date"].dt.dayofweek
  return train

In [4]:
def normalize(train):
  train = train.drop(["Date"], axis=1)
  train_norm = train.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
  return train_norm

In [5]:
def buildTrain(train, pastDay=30, futureDay=5):
  X_train, Y_train = [], []
  for i in range(train.shape[0]-futureDay-pastDay):
    X_train.append(np.array(train.iloc[i:i+pastDay]))
    Y_train.append(np.array(train.iloc[i+pastDay:i+pastDay+futureDay]["Adj Close"]))
  return np.array(X_train), np.array(Y_train)

In [6]:
def shuffle(X,Y):
  np.random.seed(10)
  randomList = np.arange(X.shape[0])
  np.random.shuffle(randomList)
  return X[randomList], Y[randomList]

In [7]:
def splitData(X,Y,rate):
  X_train = X[int(X.shape[0]*rate):]
  Y_train = Y[int(Y.shape[0]*rate):]
  X_val = X[:int(X.shape[0]*rate)]
  Y_val = Y[:int(Y.shape[0]*rate)]
  return X_train, Y_train, X_val, Y_val

In [8]:
# read SPY.csv
train = readTrain()

# Augment the features (year, month, date, day)
train_Aug = augFeatures(train)

# Normalization
train_norm = normalize(train_Aug)

# build Data, use last 30 days to predict next 5 days
X_train, Y_train = buildTrain(train_norm, 30, 5)

# shuffle the data, and random seed is 10
X_train, Y_train = shuffle(X_train, Y_train)

# split training data and validation data
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)


In [9]:
def buildOneToOneModel(shape):
  model = Sequential()
  model.add(LSTM(10, input_length=shape[1], input_dim=shape[2],return_sequences=True))
  # output shape: (1, 1)
  model.add(TimeDistributed(Dense(1)))    # or use model.add(Dense(1))
  model.compile(loss="mse", optimizer="adam")
  model.summary()
  return model

In [10]:
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 1, 1)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,np.newaxis]
Y_val = Y_val[:,np.newaxis]

model = buildOneToOneModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, return_sequences=True, input_shape=(1, 10))`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 10)             840       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 1, 1)              11        
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________
Train on 5710 samples, validate on 634 samples
Epoch 1/1000
5710/5710 [==============================] - 1s 221us/step - loss: 0.0720 - val_loss: 0.0531
Epoch 2/1000
5710/5710 [==============================] - 0s 11us/step - loss: 0.0424 - val_loss: 0.0289
Epoch 3/1000
5710/5710 [==============================] - 0s 11us/step - loss: 0.0202 - val_loss: 0.0114
Epoch 4/1000
5710/5710 [==============================] - 0s 11us/step - loss: 0.0065 - val_loss: 0.0027
Epoch 5/1000
5710/5710 [==============================] - 0s 11us/step -

5710/5710 [==============================] - 0s 11us/step - loss: 3.7494e-05 - val_loss: 3.9329e-05
Epoch 68/1000
5710/5710 [==============================] - 0s 11us/step - loss: 3.7134e-05 - val_loss: 3.9520e-05
Epoch 69/1000
5710/5710 [==============================] - 0s 11us/step - loss: 3.6952e-05 - val_loss: 3.8466e-05
Epoch 70/1000
5710/5710 [==============================] - 0s 10us/step - loss: 3.6628e-05 - val_loss: 3.8006e-05
Epoch 71/1000
5710/5710 [==============================] - 0s 11us/step - loss: 3.6206e-05 - val_loss: 3.8843e-05
Epoch 72/1000
5710/5710 [==============================] - 0s 10us/step - loss: 3.6181e-05 - val_loss: 3.7545e-05
Epoch 73/1000
5710/5710 [==============================] - 0s 11us/step - loss: 3.5745e-05 - val_loss: 3.7271e-05
Epoch 74/1000
5710/5710 [==============================] - 0s 10us/step - loss: 3.5141e-05 - val_loss: 3.7086e-05
Epoch 75/1000
5710/5710 [==============================] - 0s 10us/step - loss: 3.4982e-05 - val_loss:

5710/5710 [==============================] - 0s 12us/step - loss: 2.4515e-05 - val_loss: 2.3682e-05
Epoch 139/1000
5710/5710 [==============================] - 0s 12us/step - loss: 2.4262e-05 - val_loss: 2.3706e-05
Epoch 140/1000
5710/5710 [==============================] - 0s 12us/step - loss: 2.4301e-05 - val_loss: 2.3777e-05
Epoch 141/1000
5710/5710 [==============================] - 0s 12us/step - loss: 2.4421e-05 - val_loss: 2.3606e-05
Epoch 142/1000
5710/5710 [==============================] - 0s 12us/step - loss: 2.4325e-05 - val_loss: 2.3897e-05
Epoch 143/1000
5710/5710 [==============================] - 0s 12us/step - loss: 2.3925e-05 - val_loss: 2.3474e-05
Epoch 144/1000
5710/5710 [==============================] - 0s 10us/step - loss: 2.3821e-05 - val_loss: 2.2932e-05
Epoch 145/1000
5710/5710 [==============================] - 0s 10us/step - loss: 2.3922e-05 - val_loss: 2.2983e-05
Epoch 146/1000
5710/5710 [==============================] - 0s 12us/step - loss: 2.3878e-05 - v